<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_nlp2.0/blob/week_5_scraping_parsing/ria_parsing_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сбор данных с сайта РИА Новости

*Ноутбук сделан по мотивам ноутбука авторства Марии Гольдштейн и Кирилла Долматова.*

Установим и импортируем необходимые библиотеки.

In [1]:
%%capture
!pip install selenium

In [2]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

Далее зададим необходимые константы.

`DEPTH` - число страниц для скрапинга

In [3]:
SLEEP = 2
DEPTH = 20
BASE_URL = "https://ria.ru/"
TOPICS = ["economy", "society", "incidents", "science", "culture"]

In [4]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [5]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

Посмотрим на функцию для скачивания страниц.

In [6]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            # push to list 20 next articles
            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

Посмотрим на функцию для парсинга скачанных страниц.

In [7]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

Скачаем новости (количество регулируется константой DEPTH выше).

In [8]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 5/5 [02:17<00:00, 27.49s/it]


Сохраним результат в файл.

In [9]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

## Вопрос 1

Сколько страниц `pages` мы спарсили?

In [10]:
len(pages)

1488

Распарсим каждую страницу и достанем из нее необходимые атрибуты:

* в цикле примените функцию `parse_page` к каждой странице и сохраните результат в объект `res`

* затем добавьте `res` в список `data`


Для скорости распарсим только первые 100 страниц.

In [12]:
data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages[:100])):
    try:
        res = parse_page(page)
        data.append(res)

        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 100/100 [07:40<00:00,  4.61s/it]


Преобразуем в датафрейм полученные данные.

## Вопрос 2

Сколько столбцов в таблице `df`?

In [13]:
df = pd.DataFrame(data=data)

df.shape

(100, 6)

Добавьте в таблицу `df` столбец `topic`, содержащий список `topics_order_fixed`

In [16]:
len(topics_order_fixed)

100

In [17]:
df["topic"] = topics_order_fixed

## Вопрос 3

Какой `topic` имеет новость из первой строки таблицы?

In [18]:
df.head()

,id,url,title,subtitle,content,datetime,topic
0,1916472715,https://ria.ru/20231218/podmoskove-1916472715....,Подмосковье и Белоруссия могут реализовывать и...,Воробьев: Подмосковье и Белоруссия могут реали...,"МИНСК, 18 дек – РИА Новости. Белоруссия и Моск...",13:17 18.12.2023,economy
1,1916465345,https://ria.ru/20231218/smartfon-1916465345.html,"В России представили ""антишпионский"" смартфон","""РОСА"" представила ""антишпионский"" смартфон ""Р...","МОСКВА, 18 дек - РИА Новости. Научно-техническ...",13:02 18.12.2023,economy
2,1916464744,https://ria.ru/20231218/peskov-1916464744.html,Песков прокомментировал небыстрое снижение цен...,Песков назвал небыстрое снижение цен на яйца и...,"МОСКВА, 18 дек - РИА Новости. Небыстрое снижен...",13:01 18.12.2023,economy
3,1916460082,https://ria.ru/20231218/bolgariya-1916460082.html,Парламент Болгарии одобрил ограничения для НПЗ...,"Парламент Болгарии, несмотря на протесты, одоб...","МОСКВА, 18 дек - РИА Новости. Народное собрани...",12:55 18.12.2023,economy
4,1916450010,https://ria.ru/20231218/dyadya_vanya-191645001...,"Товары ""Дядя Ваня"" завоевывают мир со знаком к...",,"МОСКВА, 18 дек - РИА Новости. Производитель ма...",12:22 18.12.2023,economy


Сохраним скачанные данные в файл.

In [19]:
df.to_pickle("df_ria.p", compression="gzip")

## Вопрос 4

Есть ли пропуски в каких-либо столбцах в таблице `df`?

In [22]:
df.isna().any().any()

False